In [ ]:
import pandas as pd 
import numpy as np 


## Phân tích cấu trúc tài chính 

In [ ]:
# Financing Ratio (Hệ số tài trợ)
def equity_ratio(equity, total_assets):
    return equity / total_assets

#  Long-Term Asset Self-Financing Ratio (Hệ số tự tài trợ tài sản dài hạn)
# longterm_assets = BS_200
def long_term_asset_self_financing_ratio(permanent_capital, long_term_assets):
    return permanent_capital / long_term_assets

#Fixed Asset Self-Financing Ratio (Hệ số tự tài trợ tài sản cố định - TSCĐ)
# permanent capital = VCSH(400)+ Nợ dài hạn (330)
# fixed assets = TSCĐ đã và đang đầu tư(220 + 240/242)
def fixed_asset_self_financing_ratio(permanent_capital, fixed_assets):
    return permanent_capital / fixed_assets

# General Solvency Ratio (Hệ số khả năng thanh toán tổng quát) 
def general_solvency_ratio(total_liabilities, total_assets):
    return total_assets / total_liabilities

# ROI 
def return_on_investment(net_profit, total_investment):
    return net_profit / total_investment

# ROIC (Return on Invested Capital)
#Net Operating Profit After Taxes (NOPAT) = 60
def ROIC (NOPAT, invested_capital):
    return NOPAT / invested_capital

# Return on Long-Term Capital
# Earnings Before Interest and Taxes (EBIT) (Lợi nhuận trước thuế và lãi vay) = 50
def return_on_long_term_capital(EBIT, average_long_term_capital):
    return EBIT / average_long_term_capital

# Basic Earning Power (BEP) (Năng lực sinh lời cơ bản)
def basic_earning_power(EBIT, average_total_assets):
    return EBIT / average_total_assets

#Debt-to-Assets Ratio
def debt_to_assets_ratio(total_liabilities, total_assets):
    return total_liabilities / total_assets

# Debt-to-Equity Ratio
def debt_to_equity_ratio(total_liabilities, equity):
    return total_liabilities / equity

# Short-Term Debt-to-Assets Ratio
def short_term_debt_to_assets_ratio(short_term_liabilities, total_assets):
    return short_time_liabilities / total_assets

#Equity-to-Total Assets Ratio:
def equity_to_total_assets_ratio(equity, total_assets):
    return equity / total_assets

#Interest Coverage Ratio:
def interest_coverage_ratio(EBIT, interest_expense):
    return EBIT / interest_expense
# Long-Term Debt-to-Equity Ratio
#longterm liabilities = BS_330
def long_term_debt_to_equity_ratio(long_term_liabilities, equity):
    return long_term_liabilities / equity

# Short-Term Debt-to-Total Assets Ratio:
def short_term_debt_to_total_assets_ratio(short_term_liabilities, total_assets):
    return short_term_liabilities / total_assets



## Phân tích khả năng thanh toán (LIquidity)

In [ ]:
# Receivables-to-Payables Ratio (Tỷ lệ giữa nợ phải thu và nợ phải trả)
# accounts_receivable(130+210) / total_liabilities(310+330)
def receivables_to_payables_ratio(accounts_receivable, total_liabilities):
    return accounts_receivable / total_liabilities

#Receivables-to-Total Assets Ratio
def receivables_to_total_assets_ratio(accounts_receivable, total_assets):
    return accounts_receivable / total_assets

#Debt-to-Total Capital Ratio 
# total_liabilities = BS_300
# total_capital = BS_440
def debt_to_total_capital_ratio(total_liabilities, total_capital):
    return total_liabilities / total_capital

# the Receivables-to-Sales Ratio (Tỷ lệ giữa nợ phải thu người mua so với tổng số tiền hàng bán ra trong kỳ) 
# accouts_receivables(131+211) / total_sales (10)
def receivables_to_sales_ratio(accounts_receivables, total_sales):
    return accounts_receivables / total_sales

# Allowance for Doubtful Accounts Ratio (Tỷ lệ giữa dự phòng nợ phải thu khó đòi so với tổng số nợ phải thu người mua)
# allowance_for_doubtful_accounts(137+219) / accounts_receivables(131+211)
def allowance_for_doubtful_accounts_ratio(allowance_for_doubtful_accounts, accounts_receivables):
    return allowance_for_doubtful_accounts / accounts_receivables

#Asset-to-Debt Ratio
def asset_to_debt_ratio(total_assets, total_liabilities):
    return total_assets / total_liabilities

# Current ratio (Hệ số thanh khoản ngắn hạn)
# current_assets = BS_100 - BS_151
def current_ratio(current_assets, current_liabilities):
    return current_assets / current_liabilities

# quick ratio (Hệ số thanh khoản nhanh)
# current assets - inventory (BS_100 - BS_151 - BS_140) / current liabilities
def quick_ratio(current_assets, inventory, current_liabilities):
    return (current_assets - inventory) / current_liabilities

# Cash Ratio (Hệ số thanh khoản tiền mặt)
# cash_and_cash_equivalents(BS_110) / current_liabilities
def cash_ratio(cash_and_cash_equivalents, current_liabilities):
    return cash_and_cash_equivalents / current_liabilities

# Current Maturity Coverage Ratio= Cash and Cash Equivalents/ Current Maturing Debt 
# cash_and_cash_equivalents(BS_110) / current_maturing_debt ( TMBCTC)

# long_term_debt_coverage_ratio = non_current_assets / long_term_debt
def long_term_debt_coverage_ratio(non_current_assets, non_current_liabilities):
    return non_current_assets / non_current_liabilities
# Debt-to-Equity Ratio = Total Liabilities/ Total Equity
def debt_to_equity_ratio(total_liabilities, total_equity):
    return total_liabilities / total_equity

# Long-Term Debt to Equity Capital Ratio
# non_current_liabilities = BS_330
def long_term_debt_to_equity_capital_ratio(non_current_liabilities, equity):
    return non_current_liabilities / equity

# Time Interest Earned (TIE) = EBIT(IS_050)/ Interest Expense(IS_023)
def time_interest_earned(EBIT, interest_expense):
    return EBIT / interest_expense

# Debt to Tangible Net Worth Ratio = total_liabilities / equity - intangible_assets (BS_227)
def debt_to_tangible_net_worth_ratio(total_liabilities, equity, intangible_assets):
    return total_liabilities / (equity - intangible_assets)



 


## Phân tích rủi ro tài chính (Financial risk)

In [ ]:
# financial_leverage = total_liabilities(BS_300) / total_lia_and_equity (BS_440)
# 0< financial_leverage < 1 : Close to 0 is better
def financial_leverage(total_liabilities, total_lia_and_equity):
    return total_liabilities / total_lia_and_equity

# Allowance for Doubtful Accounts to Total Assets Ratio = allowance_for_doubtful_accounts(BS_137+BS_219)/ total_assets (BS_270)
def allowance_for_doubtful_accounts_to_total_assets_ratio(allowance_for_doubtful_accounts, total_assets):
    return allowance_for_doubtful_accounts / total_assets

# Hệ số tài trợ thường xuyên (Permanent Financing Ratio) = permanent_capital(BS_400+BS_330) / total_lia_and_equity (BS_440)
def permanent_financing_ratio(permanent_capital, total_lia_and_equity):
    return permanent_capital / total_lia_and_equity





## phân tích kết quả kinh doanh 

In [ ]:
# Doanh thu thuần về bán hàng và cung cấp dịch vụ ,IS_010
# Doanh thu hoạt động tài chính,IS_021,Financial income
# doanh thu thuần = IS_010 + IS_021

# Financial Income to Net Revenue Ratio = Financial Income(IS_021)/ Net Revenue
def financial_income_to_net_revenue_ratio(financial_income, net_revenue):
    return financial_income / net_revenue



## phân tích khả năng sinh lời ( profitability )

In [ ]:
# ROA : Return on Assets = net_income(IS_060) / total_assets
def return_on_assets(net_income, total_assets):
    return net_income / total_assets

#Return on fixed assets = net_income(IS_060) / average_fixed_assets (BS_220)
def return_on_fixed_assets(net_income, average_fixed_assets):
    return net_income / average_fixed_assets

#RNOA= Net Operating Profit After Tax (NOPAT)/Average Net Operating Assets
def RNOA(NOPAT, average_net_operating_assets):
    return NOPAT / average_net_operating_assets

# return on long-term operating assets = net_income(IS_060) / average_long_term_operating_assets
def return_on_long_term_operating_assets(net_income, average_long_term_operating_assets):
    return net_income / average_long_term_operating_assets

# SỨC SINH LỢI CƠ BẢN CỦA TÀI SẢN : BEPR (Basic Earning Power Ratio) = EBIT(IS_050) / total_assets
def BEPR(EBIT, total_assets):
    return EBIT / total_assets

#ROE = Net Income/  Equity
def ROE(net_income, equity):
    return net_income / equity

# Return on Common Equity (sức sinh lời của vốn cổ phần thường) = Net Income (IS_060) - (Preferred Dividends-CF-036)/ Average Common Equity
def return_on_common_equity(net_income, preferred_dividends, average_common_equity):
    return net_income - preferred_dividends / average_common_equity

# Profitability of Cost of Goods Sold (COGS) = net income from operating(IS_030)/ COGS (IS_011)
def profitability_of_cost_of_goods_sold(net_income_from_operating, COGS):
    return net_income_from_operating / COGS

# Hệ số chênh lệch giá (Price Spread Ratio): gross_profit(IS_020) / COGS(IS_011)
def price_spread_ratio(gross_profit, COGS):
    return gross_profit / COGS

# Sức sinh lợi của chi phí kinh doanh (Profitability of Operating Expenses): net_income_from_operating(IS_030) / total_operating_expenses(IS_025+IS_026+IS_011)
def profitability_of_operating_expenses(net_income_from_operating, total_operating_expenses):
    return net_income_from_operating / total_operating_expenses

# ROS = Net Income/ Net Sales (IS_010)
def ROS(net_income, net_sales):
    return net_income / net_sales

# adjusted ROS = net_income(IS_060) - cổ tức ưu đãi/ net_sales(IS_010) 

# Operating Profit Margin (OPM) = Net profit from operating activities (IS_030)/ Net Sales
def net_operating_profit_margin(net_profit_from_operating, net_sales):
    return net_profit_from_operating / net_sales

# Gross Profit Margin (GPM) = Gross Profit(IS_020)/ Net Sales
def gross_profit_margin(gross_profit, net_sales):
    return gross_profit / net_sales

#net operating profit margin = lợi nhuận sau thuế từ hđkd(net operating profit after tax, NOPAT)/ net sales(IS_010)
# NOPAT = lợi nhuận trc thuế từ hđkd x (1 - thuế suất)
def net_operating_profit_margin(NOPAT, net_sales):
    return NOPAT / net_sales

# price to book ratio = giá cổ phiếu / giá trị sổ sách ( market price per share/book value per share)
# price to earnings ratio = giá cổ phiếu / lợi nhuận trên cổ phiếu
#Dividend Payout Ratio =  Dividend per Share/ Earnings per Share
# Dividend Yield = Dividend per Share/ Market Price per Share





## phân tích cashflow


In [ ]:
# Cash Flow Adequacy Ratio = Cash Flow from Operations/ Capital_Expenditures+Debt_Repayments+Dividends_Paid
# Free Cash Flow (FCF)=Operating_Cash_Flow (CF_020) − Capital_Expenditures_(CAPEX) ( CF_021+CF_023) - Chi trả cổ tức (CF_036)
def free_cash_flow(operating_cash_flow, capital_expenditures, dividends_paid):
    return operating_cash_flow - capital_expenditures - dividends_paid

# free cash flow to the operating cash flow ratio = free_cash_flow / operating_cash_flow (CF_020)
def free_cash_flow_to_operating_cash_flow_ratio(free_cash_flow, operating_cash_flow):
    return free_cash_flow / operating_cash_flow

# 